If you're using this simulator, then fuck yeah. That's pretty cool. Anyway, it's really easy to use. If you want to do some tests, you can totally ignore the first cell (below this) and skip straight to the 2nd one. I suggest reading the documentation and a bit of the code to understand what exactly is happening. Scroll down to the next Markdown cell to see more instructions about how to use this program. Run each cell in this notebook before proceeding to the next one.

In [ ]:
# Speedrun Encounter Mean Completion Time Simulator (SEMCOT Simulator)
# Created by ScarlettTheHuman July 08, 2020
# Last updated: July 08, 2020

import numpy as np
import math
from random import random

"""
    Summary:
    
    This simulation will determine the mean time to get through a speedrun
    encounter (designed for The Last of Us Part 2). It takes into account
    all the possible locations an attempt might fail by using user-input
    data. It assumes all failure points operate on a simple pass/fail basis
    and treats every subsequent failure point as statistically dependent on
    the success of the previous point. It also treats each time and
    probability of possible failure as fixed, which simplifies the inter-
    relations between speedrunning RNG-heavy sections and human error.
    
    By running a simulation of an encounter over many iterations, the user
    can get an idea of how much time they may lose due to potential failures.
    
    
    Variables:
    
    Specified by the user:
    time_success (float): The time it takes to complete the strat in a fully
        successful attempt (seconds).
    fail_points (array of shape (n*, 2) with floats):
        For each array, the first element represents the time (in seconds)
        at which a strat may fail (from a RE/RC). The second element
        represents the probability (between 0 and 1, exclusive) that the
        strat SUCCEEDS at that point.
        *n represents the number of failure points, which is not explicitly
        defined by the user. The user only needs to insert as many
        coordinates as they want.

"""

class StopExecution(Exception):
    
    """
    A class to terminate the program without killing the kernal in case of
    error. Useful if user is using IPython.
    """
    
    def _render_traceback_(self):
        pass


def get_variables(time_success, fail_points):
    
    """
    Takes the user-input variables and gets things set up to run the
    simulation.
    
    Function Variables:
    time_success (float): The time it takes to complete the strat in a fully
        successful attempt.
    fail_points (array of floats): The set of coordinates that correspond
        to the time of failure and probability of SUCCESS at a given
        potential failure point.
    
    Returns:
    fail_times (array of floats): an array of failure times, sequenced
        in chronological order.
    fail_probabilities (array of floats): an array of probabilities that
        correspond to the SUCCESS of a potential fail point, sequenced
        in correspondence to their time counter-parts in chronological
        order.
    """
    
    # Initialize variables for checks and to return the appropriate data.
    
    fail_num = fail_points.shape[0]  #number of failure points in the encounter.
    time_temp = 0.
    probability_temp = 0.
    fail_times = np.zeros(fail_num)
    fail_probabilities = np.zeros(fail_num)
    idx = np.zeros(fail_num, dtype=int)
    
    # Check to make sure the user-specified times and probabilities are
    # within the appropriate range.
    
    abort = False
    for i in range(fail_num):
        idx[i] = i
        time_temp = fail_points[i][0]
        probability_temp = fail_points[i][1]
        
        if (time_temp < 0.):
            print('One or more of your times is less than 0. Fix this',
                 'and try again.','/n')
            abort = True
        if (time_temp > time_success):
            print('One or more of your times is greater than the total',
                 'strat completion time. Fix this and try again.')
            abort = True
        if (probability_temp <= 0.) or (probability_temp > 1.):
            print('One or more of your probabilities is outside the',
                 'range of (0,1]. Fix this and try again.','/n')
            abort = True
        
        # If there is a user-input error, end the program.
        if abort == True:
            raise StopExecution
        
        # Fill the arrays with fail time and probability data
        fail_times[i] = time_temp
        fail_probabilities[i] = probability_temp
        
    
    # If the user did not input failure times in order, re-order the
    # corresponding time and probability arrays.
    
    for i in range(fail_num):
        for j in range(fail_num - i - 1):
            
            if fail_times[i] > fail_times[i+j+1]:
                
                time_temp = fail_times[i]
                fail_times[i] = fail_times[i+j+1]
                fail_times[i+j+1] = time_temp
                
                probability_temp = fail_probabilities[i]
                fail_probabilities[i] = fail_probabilities[i+j+1]
                fail_probabilities[i+j+1] = probability_temp
                
    return(fail_times, fail_probabilities)


def simulate_encounter(num_iterations, time_success, fail_times,
                    fail_probabilities, percentiles=None,
                    print_percentiles=True, outlook=None):
    """
    Runs a series of encounter simulations.
    
    Variables:
    num_iterations (int): The number of encounters you want to simulate.
    time_success (float): The time it takes to complete the strat in a
        fully successful attempt.
    fail_times (array of floats): an array of failure times, sequenced
        in chronological order.
    fail_probabilities (array of floats): an array of probabilities that
        correspond to the SUCCESS of a potential fail point, sequenced
        in correspondence to their time counter-parts in chronological
        order.

    Returns nothing. Prints the mean time to complete the strat and the
    mean number of failures.    
    """
    
    # Initialize the variables for the simulation.
    fail_num = len(fail_times)
    time_cum = np.zeros(num_iterations)
    failure_count = np.zeros(num_iterations, dtype=int)
    
    # Run the simulation.
    for i in range(num_iterations):
        j = 0
        while j < fail_num:
            random_probability = random()

            # If you pass this stage, proceed.
            if random_probability < fail_probabilities[j]:
                j += 1
                
            # If you fail at this point, count the time and go back to the
            # beginning.
            else:
                time_cum[i] = time_cum[i] + fail_times[j]
                failure_count[i] += 1
                j = 0

        # Add the time of a full successful attempt
        time_cum[i] = time_cum[i] + time_success
    
    # Determine the mean time of completion and mean number of failures
    mean_time = np.mean(time_cum)
    mean_failures = np.mean(failure_count)
    
    # Find the percentile times
    if type(percentiles) == None:
        perc = np.array([50, 80, 95, 99])
    else:
        perc = percentiles
        
    # Print the results of the simulation.
    mean_time_min = math.floor(mean_time / 60.)
    mean_time_sec = round(mean_time-60.*mean_time_min, 1)
    if mean_time_sec < 10.:
        mean_time_str = str(mean_time_min) + ':0' + str(mean_time_sec)
    else:
        mean_time_str = str(mean_time_min) + ':' + str(mean_time_sec)
    print('It will take an average of', mean_time_str, 'to finish',
          'this encounter, due to an average of', mean_failures,
          'failures per encounter.')
        
    # Print the percentile scores if desired.
    if print_percentiles:
        print('\n')
        perc_indexes = perc*num_iterations/100
        time_cum.sort()
        for i in range(len(percentiles)):
            
            perc_time = time_cum[int(perc_indexes[i])]
            perc_time_min = math.floor(perc_time / 60.)
            perc_time_sec = round(perc_time-60.*perc_time_min, 1)
            if perc_time_sec < 10.:
                perc_time_str = str(perc_time_min) + ':0' + str(perc_time_sec)
            else:
                perc_time_str = str(perc_time_min) + ':' + str(perc_time_sec)
            
            
            if outlook=='optimistic':
                print_perc = str(perc[i]) + '%'
                print(print_perc, 'of encounters were finished in',
                      perc_time_str, 'or less.')
            else:
                print_perc = str(100-perc[i]) + '%'
                print(print_perc, 'of encounters took', perc_time_str,
                      'or longer to finish.')


Did you run the cell above this? Good. Now that we have all the functions and stuff set up, we can do a test. Here's an example influenced by my recent strat practice in TLOU2 at The Resort (survivor). (Don't change anything in the next cell. Just look at how it works.)To explain why I chose the data that I did:

I estimated that it took around 80 seconds to beat on a successful attempt, although this number does vary a little. I also found that I would:
* get alerted sometimes around 16 seconds in.
* miss my headshots and get alerted/grabbed around 23 seconds in.
* get shotgunned on the stairs a lot around 30 seconds in.
* sometimes get shot and killed as late as 38 seconds in.
* and sometimes get shot and killed around 52 seconds in.

Go ahead and run the next cell to see what happens.

In [ ]:
# Define how long it takes to make it through a successful attempt.
time_success = 80.

# Fill the failure point array with time/probability pairs. The first element
# of each pair represents a time in seconds, and the second element represents
# the probability of SUCCESSFULLY getting through that part of the encounter.
fail_points = np.array([
                            [16. ,   0.85],
                            [23. ,   0.95],
                            [30. ,   0.70],
                            [38. ,   0.80],
                            [52. ,   0.90]
                      ])

# Call the get_variables function to retrieve the necessary data.
fail_times, fail_probabilities = get_variables(time_success, fail_points)

# How many encounters should we simulate? (NOTE: I suggest leaving this number
# alone. It should be at least 100,000 for a reliable result.)
num_iterations = 100000

# Set a custom series of percentile checks to display. Failing to call
# percentile will simply result in the default choices of (50%, 80%, 95%, 99%).
percentiles = np.array([50, 80, 95, 99])

# Display the percentile times at the end of simulation.
print_percentiles = True
outlook = 'optimistic'
#outlook = 'pessimistic'

# Let's run the simulation!
simulate_encounter(num_iterations, time_success, fail_times, fail_probabilities,
                   percentiles, print_percentiles, outlook)

Now it's your turn to run the simulation! You will need to do something at the spots you see a '###'. You don't have to change anything else. If you also want to change the parameters of the simulation, knock yourself out. Let Scarlett know if you have any suggestions. And definitely share your data with her!! You can hang out with her on Twitch at https://twitch.tv/ScarlettTheHuman.

In [ ]:
# Define how long it takes to make it through a successful attempt
time_success =   ### PUT SOMETHING HERE.

# Fill the failure point array with time/probability pairs. The first element
# of each pair represents a time in seconds, and the second element represents
# the probability of SUCCESSFULLY getting through that part of the encounter.
fail_points = np.array([
                            [  ,  ],  ### PUT SOMETHING HERE and add more!
                      ])

# Call the get_variables function to retrieve the necessary data.
fail_times, fail_probabilities = get_variables(time_success, fail_points)

# How many encounters should we simulate? (NOTE: I suggest leaving this number
# alone. It should be at least 100,000 for a reliable result.)
num_iterations = 100000

# Set a custom series of percentile checks to display. Failing to call
# percentile will simply result in the default choices of (50%, 80%, 95%, 99%).
percentiles = np.array([50, 80, 95, 99])  ### Adjust if you want to.

# Displays the percentile times at the end of simulation.
print_percentiles = True
#outlook = 'optimistic'   ### Uncomment one of
#outlook = 'pessimistic'  ### these options.

# Let's run the simulation!
simulate_encounter(num_iterations, time_success, fail_times, fail_probabilities,
                   percentiles, print_percentiles, outlook)